# 1 class_decorators

In [1]:
def my_class_decorator(cls):
    for name, attr in vars(cls).items():
        print(name)
    return cls

@my_class_decorator
class Temperature:
    
    def __init__(self, kelvin):
        self._kelvin = kelvin
    
    def get_kelvin(self):
        return self._kelvin
    
    def set_kelvin(self, value):
        self._kelvin = value

__module__
__init__
get_kelvin
set_kelvin
__dict__
__weakref__
__doc__


# 2 Enforcing Class Invariants

In [2]:
import functools


def invariant(predicate):
    """Create a class decorator which checks a class invariant.

    Args:
        predicate: A callable to which, after every method invocation,
            the object on which the method was called will be passed.
            The predicate should evaluate to True if the class invariant
            has been maintained, or False if it has been violated.

    Returns:
        A class decorator for checking the class invariant tested by
        the supplied predicate function.
    """
    def invariant_checking_class_decorator(cls):
        """A class decorator for checking invariants."""

        method_names = [name for name, attr in vars(cls).items() if callable(attr)]
        for name in method_names:
            _wrap_method_with_invariant_checking_proxy(cls, name, predicate)

        return cls

    return invariant_checking_class_decorator


def _wrap_method_with_invariant_checking_proxy(cls, name, predicate):
    method = getattr(cls, name)
    assert callable(method)

    @functools.wraps(method)
    def invariant_checking_method_decorator(self, *args, **kwargs):
        result = method(self, *args, **kwargs)
        if not predicate(self):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
        return result

    setattr(cls, name, invariant_checking_method_decorator)


def not_below_absolute_zero(temperature):
    """Temperature not below absolute zero"""
    return temperature._kelvin >= 0
    

@invariant(not_below_absolute_zero)
class Temperature:

    def __init__(self, kelvin):
        self._kelvin = kelvin

    def get_kelvin(self):
        return self._kelvin

    def set_kelvin(self, value):
        self._kelvin = value

In [3]:
t = Temperature(5.0)

In [5]:
# t = Temperature(-1.0)

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-4-b731ade64207> in <module>
# ----> 1 t = Temperature(-1.0)

# <ipython-input-2-56e08b4ba38b> in invariant_checking_method_decorator(self, *args, **kwargs)
#      35         result = method(self, *args, **kwargs)
#      36         if not predicate(self):
# ---> 37             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
#      38         return result
#      39 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x0000024021FA7E48>

In [6]:
s = Temperature(42.0)

In [8]:
# s.set_kelvin(-1.0)

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-7-8a13266ab093> in <module>
# ----> 1 s.set_kelvin(-1.0)

# <ipython-input-2-56e08b4ba38b> in invariant_checking_method_decorator(self, *args, **kwargs)
#      35         result = method(self, *args, **kwargs)
#      36         if not predicate(self):
# ---> 37             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
#      38         return result
#      39 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x0000024021FA7940>

### Adding @property

In [9]:
import functools


def invariant(predicate):
    """Create a class decorator which checks a class invariant.

    Args:
        predicate: A callable to which, after every method invocation,
            the object on which the method was called will be passed.
            The predicate should evaluate to True if the class invariant
            has been maintained, or False if it has been violated.

    Returns:
        A class decorator for checking the class invariant tested by
        the supplied predicate function.
    """

    def invariant_checking_class_decorator(cls):
        """A class decorator for checking invariants."""

        method_names = [name for name, attr in vars(cls).items() if callable(attr)]
        for name in method_names:
            _wrap_method_with_invariant_checking_proxy(cls, name, predicate)

        return cls

    return invariant_checking_class_decorator


def _wrap_method_with_invariant_checking_proxy(cls, name, predicate):
    method = getattr(cls, name)
    assert callable(method)

    @functools.wraps(method)
    def invariant_checking_method_decorator(self, *args, **kwargs):
        result = method(self, *args, **kwargs)
        if not predicate(self):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
        return result

    setattr(cls, name, invariant_checking_method_decorator)


def not_below_absolute_zero(temperature):
    """Temperature not below absolute zero"""
    return temperature._kelvin >= 0


@invariant(not_below_absolute_zero)
class Temperature:

    def __init__(self, kelvin):
        self._kelvin = kelvin

    def get_kelvin(self):
        return self._kelvin

    def set_kelvin(self, value):
        self._kelvin = value

    @property
    def celsius(self):
        return self._kelvin - 273.15

    @celsius.setter
    def celsius(self, value):
        self._kelvin = value + 273.15

    @property
    def fahrenheit(self):
        return self._kelvin * 9 / 5 - 459.67

    @fahrenheit.setter
    def fahrenheit(self, value):
        self._kelvin = (value + 459.67) * 5 / 9

In [14]:
t = Temperature(42.0)

In [15]:
t.celsius

-231.14999999999998

In [16]:
t.celsius = -100

In [17]:
t.get_kelvin()

173.14999999999998

In [18]:
# 温度过低应该抛异常
t.celsius = -300

In [20]:
# t.get_kelvin()

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-19-2304b01aaa7d> in <module>
# ----> 1 t.get_kelvin()

# <ipython-input-9-fb68a40da297> in invariant_checking_method_decorator(self, *args, **kwargs)
#      36         result = method(self, *args, **kwargs)
#      37         if not predicate(self):
# ---> 38             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
#      39         return result
#      40 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x000002402205A198>

# 3 Detecting and Wrapping Properties

In [21]:
from pprint import pprint as pp

In [22]:
pp(vars(Temperature))

mappingproxy({'__dict__': <attribute '__dict__' of 'Temperature' objects>,
              '__doc__': None,
              '__init__': <function Temperature.__init__ at 0x00000240220369D8>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Temperature' objects>,
              'celsius': <property object at 0x0000024022059278>,
              'fahrenheit': <property object at 0x00000240220592C8>,
              'get_kelvin': <function Temperature.get_kelvin at 0x00000240220361E0>,
              'set_kelvin': <function Temperature.set_kelvin at 0x0000024022036598>})


In [23]:
import functools


def invariant(predicate):
    """Create a class decorator which checks a class invariant.

    Args:
        predicate: A callable to which, after every method invocation,
            the object on which the method was called will be passed.
            The predicate should evaluate to True if the class invariant
            has been maintained, or False if it has been violated.

    Returns:
        A class decorator for checking the class invariant tested by
        the supplied predicate function.
    """

    def invariant_checking_class_decorator(cls):
        """A class decorator for checking invariants."""

        method_names = [name for name, attr in vars(cls).items() if callable(attr)]
        for name in method_names:
            _wrap_method_with_invariant_checking_proxy(cls, name, predicate)

        property_names = [name for name, attr in vars(cls).items() if isinstance(attr, property)]
        for name in property_names:
            _wrap_property_with_invariant_checking_proxy(cls, name, predicate)

        return cls

    return invariant_checking_class_decorator


def _wrap_method_with_invariant_checking_proxy(cls, name, predicate):
    method = getattr(cls, name)
    assert callable(method)

    @functools.wraps(method)
    def invariant_checking_method_decorator(self, *args, **kwargs):
        result = method(self, *args, **kwargs)
        if not predicate(self):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
        return result

    setattr(cls, name, invariant_checking_method_decorator)


def _wrap_property_with_invariant_checking_proxy(cls, name, predicate):
    prop = getattr(cls, name)
    assert isinstance(prop, property)

    invariant_checking_proxy = InvariantCheckingPropertyProxy(prop, predicate)

    setattr(cls, name, invariant_checking_proxy)


class InvariantCheckingPropertyProxy:

    def __init__(self, referent, predicate):
        self._referent = referent
        self._predicate = predicate

    def __get__(self, instance, owner):
        if instance is None:
            return self._referent
        result = self._referent.__get__(instance, owner)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result

    def __set__(self, instance, value):
        result = self._referent.__set__(instance, value)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result

    def __delete__(self, instance):
        result = self._referent.__delete__(instance)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result


def not_below_absolute_zero(temperature):
    """Temperature not below absolute zero"""
    return temperature._kelvin >= 0


@invariant(not_below_absolute_zero)
class Temperature:

    def __init__(self, kelvin):
        self._kelvin = kelvin

    def get_kelvin(self):
        return self._kelvin

    def set_kelvin(self, value):
        self._kelvin = value

    @property
    def celsius(self):
        return self._kelvin - 273.15

    @celsius.setter
    def celsius(self, value):
        self._kelvin = value + 273.15

    @property
    def fahrenheit(self):
        return self._kelvin * 9 / 5 - 459.67

    @fahrenheit.setter
    def fahrenheit(self, value):
        self._kelvin = (value + 459.67) * 5 / 9

In [24]:
t = Temperature(42.0)

In [26]:
# t.celsius = -300

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-25-5518b9891640> in <module>
# ----> 1 t.celsius = -300

# <ipython-input-23-f6065aade808> in __set__(self, instance, value)
#      74         if not self._predicate(instance):
#      75             raise RuntimeError("Class invariant {!r} violated for {!r}".format(
# ---> 76                 self._predicate.__doc__, instance))
#      77         return result
#      78 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x0000024023490AC8>

In [27]:
t.fahrenheit = 100

In [28]:
t.celsius

37.77777777777783

# 4 Chaining Class Decorators

In [29]:
import functools


def invariant(predicate):
    """Create a class decorator which checks a class invariant.

    Args:
        predicate: A callable to which, after every method invocation,
            the object on which the method was called will be passed.
            The predicate should evaluate to True if the class invariant
            has been maintained, or False if it has been violated.

    Returns:
        A class decorator for checking the class invariant tested by
        the supplied predicate function.
    """

    def invariant_checking_class_decorator(cls):
        """A class decorator for checking invariants."""

        method_names = [name for name, attr in vars(cls).items() if callable(attr)]
        for name in method_names:
            _wrap_method_with_invariant_checking_proxy(cls, name, predicate)

        property_names = [name for name, attr in vars(cls).items() if isinstance(attr, property)]
        for name in property_names:
            _wrap_property_with_invariant_checking_proxy(cls, name, predicate)

        return cls

    return invariant_checking_class_decorator


def _wrap_method_with_invariant_checking_proxy(cls, name, predicate):
    method = getattr(cls, name)
    assert callable(method)

    @functools.wraps(method)
    def invariant_checking_method_decorator(self, *args, **kwargs):
        result = method(self, *args, **kwargs)
        if not predicate(self):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
        return result

    setattr(cls, name, invariant_checking_method_decorator)


def _wrap_property_with_invariant_checking_proxy(cls, name, predicate):
    prop = getattr(cls, name)
    assert isinstance(prop, property)

    invariant_checking_proxy = InvariantCheckingPropertyProxy(prop, predicate)

    setattr(cls, name, invariant_checking_proxy)


class InvariantCheckingPropertyProxy:

    def __init__(self, referent, predicate):
        self._referent = referent
        self._predicate = predicate

    def __get__(self, instance, owner):
        if instance is None:
            return self._referent
        result = self._referent.__get__(instance, owner)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result

    def __set__(self, instance, value):
        result = self._referent.__set__(instance, value)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result

    def __delete__(self, instance):
        result = self._referent.__delete__(instance)
        if not self._predicate(instance):
            raise RuntimeError("Class invariant {!r} violated for {!r}".format(
                self._predicate.__doc__, instance))
        return result


def not_below_absolute_zero(temperature):
    """Temperature not below absolute zero"""
    return temperature._kelvin >= 0


def below_absolute_hot(temperature):
    """Temperature below absolute hot"""
    # See http://en.wikipedia.org/wiki/Absolute_hot
    return temperature._kelvin <= 1.416785e32


@invariant(below_absolute_hot)
@invariant(not_below_absolute_zero)
class Temperature:

    def __init__(self, kelvin):
        self._kelvin = kelvin

    def get_kelvin(self):
        return self._kelvin

    def set_kelvin(self, value):
        self._kelvin = value

    @property
    def celsius(self):
        return self._kelvin - 273.15

    @celsius.setter
    def celsius(self, value):
        self._kelvin = value + 273.15

    @property
    def fahrenheit(self):
        return self._kelvin * 9 / 5 - 459.67

    @fahrenheit.setter
    def fahrenheit(self, value):
        self._kelvin = (value + 459.67) * 5 / 9

In [30]:
t = Temperature(37.5)

In [32]:
# t.set_kelvin(-300)

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-31-83f3845eb1de> in <module>
# ----> 1 t.set_kelvin(-300)

# <ipython-input-29-ce2bc13a2614> in invariant_checking_method_decorator(self, *args, **kwargs)
#      38     @functools.wraps(method)
#      39     def invariant_checking_method_decorator(self, *args, **kwargs):
# ---> 40         result = method(self, *args, **kwargs)
#      41         if not predicate(self):
#      42             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))

# <ipython-input-29-ce2bc13a2614> in invariant_checking_method_decorator(self, *args, **kwargs)
#      40         result = method(self, *args, **kwargs)
#      41         if not predicate(self):
# ---> 42             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
#      43         return result
#      44 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x0000024023484748>

In [34]:
# t.set_kelvin(1e33)

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-33-21f1b6e7c3d6> in <module>
# ----> 1 t.set_kelvin(1e33)

# <ipython-input-29-ce2bc13a2614> in invariant_checking_method_decorator(self, *args, **kwargs)
#      40         result = method(self, *args, **kwargs)
#      41         if not predicate(self):
# ---> 42             raise RuntimeError("Class invariant {!r} violated for {!r}".format(predicate.__doc__, self))
#      43         return result
#      44 

# RuntimeError: Class invariant 'Temperature below absolute hot' violated for <__main__.Temperature object at 0x0000024023484748>

In [36]:
# t.celsius = -300

# ---------------------------------------------------------------------------
# RuntimeError                              Traceback (most recent call last)
# <ipython-input-35-5518b9891640> in <module>
# ----> 1 t.celsius = -300

# <ipython-input-29-ce2bc13a2614> in __set__(self, instance, value)
#      74         if not self._predicate(instance):
#      75             raise RuntimeError("Class invariant {!r} violated for {!r}".format(
# ---> 76                 self._predicate.__doc__, instance))
#      77         return result
#      78 

# RuntimeError: Class invariant 'Temperature not below absolute zero' violated for <__main__.Temperature object at 0x0000024023484748>

In [38]:
# 应该抛异常, 需要Abcs解决
t.celsius = 1e34